In [6]:
import numpy as np 
import matplotlib.pyplot as plt
from qdc.grating.grating_sim import GratingSim1D, blazed_mask

In [62]:
from grating_sim import GratingSim1D
import matplotlib.pyplot as plt

sim = GratingSim1D(
    Nx          = 2**14,      # grid points
    Lx          = 8e-3,       # 8 mm
    wl0         = 808e-9,     # 808 nm blaze
    Dwl         = 100e-9,     # ±50 nm span
    N_wl        = 71,         # 11 wavelengths  (odd!)
    waist       = 800e-6,      # 40 µm beam waist
    x0          = 0,
    blaze_angle = 0.15,       # blaze angle [rad]
    f           = 0.2         # lens focal length [m]
)

x_det, I_class = sim.classical_pattern()
_,     I_spdc  = sim.spdc_pattern()

fig, ax = plt.subplots(figsize=(7, 4))
ax.semilogy(x_det*1e3, I_class, label="classical incoherent sum")
ax.semilogy(x_det*1e3, I_spdc,  label="SPDC double pass")
ax.set_xlabel("detector coordinate $x'$ [mm]")
ax.set_ylabel("normalised intensity (log)")
ax.set_title("1-D blazed-grating far field — classical vs SPDC")
ax.legend()
fig.tight_layout()
fig.show()

In [44]:
from qdc.grating.grating_sim import gaussian, blazed_mask

if True:
    mask = blazed_mask(sim.x, sim.wl0, 0.085, 808e-9)
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot(np.angle(mask), '--.')
    # ax.set_xlim([0, 1400])
    fig.show()
 
if True:
    E = gaussian_field(sim.x, sim.waist, sim.x0).astype(np.complex128)  # <<<
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.plot((np.abs(E))**2)
    fig.show()

ImportError: cannot import name 'gaussian_field' from 'qdc.grating.grating_sim' (C:\code\qdc2\qdc\grating\grating_sim.py)

In [50]:
# --------------------------------------------------------------------
# TEST CELL – sanity checks for grating_sim.py
# --------------------------------------------------------------------
import numpy as np
import matplotlib.pyplot as plt

from grating_sim import (
    GratingSim1D, blazed_mask, gaussian, lens_phase, farfield
)

# ---------------- configure a *small* simulator for quick plots -----
sim = GratingSim1D(
    Nx        = 2**12,     # coarse grid is fine for inspection
    Lx        = 8e-3,
    wl0       = 808e-9,
    Dwl       = 100e-9,
    N_wl      = 11,
    waist     = 1400e-6,
    blaze_deg = 0.45,
    f         = 0.2,
)

# -------------------------------------------------------------------
# 1. Blazed-mask phase ramp for reference and detuned wavelength
# -------------------------------------------------------------------
wl_detune = sim.wl0 + 50e-9       # +50 nm for visibility
mask_ref  = blazed_mask(sim.x, sim.wl0, sim.blaze, sim.wl0)
mask_det  = blazed_mask(sim.x, sim.wl0, sim.blaze, wl_detune)

fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(np.angle(mask_ref),  label=rf"phase @ $\lambda_0$ = {sim.wl0*1e9:.0f} nm")
ax.plot(np.angle(mask_det),  label=rf"phase @ $\lambda_0$+50 nm")
ax.set_ylabel("phase [rad]")
ax.set_xlabel("array index (x grid)")
ax.set_title("Saw-tooth phase ramp (unwrapped)")
ax.legend()
fig.show()

# -------------------------------------------------------------------
# 2. Gaussian source intensity on the grating plane
# -------------------------------------------------------------------
E0 = gaussian(sim.x, sim.waist, sim.x0)
fig, ax = plt.subplots(figsize=(7, 4))
ax.plot(np.abs(E0)**2)
ax.set_ylabel(r"$|E(x)|^2$")
ax.set_xlabel("array index (x grid)")
ax.set_title("Normalised Gaussian source intensity")
fig.show()

# -------------------------------------------------------------------
# 3. Compare single-λ classical vs SPDC pair in the far field
# -------------------------------------------------------------------
# --- choose a moderate detuning so we can see the ε^2 vs ε^4 effect
wl_s = sim.wl0 + 20e-9
wl_i = sim.wl0 - 20e-9

# classical, single pass
E_class = E0 * blazed_mask(sim.x, sim.wl0, sim.blaze, wl_s)
E_class *= lens_phase(sim.x, wl_s, sim.f)
x_det_c, I_class = farfield(E_class, wl_s, sim.f, sim.kx)

# SPDC, double pass
E_spdc = E0 * blazed_mask(sim.x, sim.wl0, sim.blaze, wl_s)
E_spdc *= blazed_mask(sim.x, sim.wl0, sim.blaze, wl_i)
E_spdc *= lens_phase(sim.x, wl_i, sim.f)
x_det_s, I_spdc = farfield(E_spdc, wl_s, sim.f, sim.kx)   # detector axis uses wl_s

fig, ax = plt.subplots(figsize=(7, 4))
ax.semilogy(x_det_c*1e3, I_class / I_class.max(), label="classical, λ = λ₀+20 nm")
ax.semilogy(x_det_s*1e3, I_spdc  / I_spdc.max(),  label="SPDC, ±20 nm pair")
ax.set_xlabel("detector coordinate $x'$ [mm]")
ax.set_ylabel("normalised intensity (log)")
ax.set_title("Single-wavelength classical vs SPDC pair")
ax.set_xlim(-150, 150)
ax.legend()
fig.show()
